In [13]:
import numpy as np
import cv2
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import os
import pywt

In [2]:
img = cv2.imread("./test images/chalamettest1.jpg")
img.shape

(275, 183, 3)

In [10]:
face_cascade = cv2.CascadeClassifier("./haarcascades/haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier("./haarcascades/haarcascade_eye.xml")


In [3]:
def get_cropped_image_if_2_eyes(image_path):
    image_path = os.path.normpath(image_path)

    if not os.path.exists(image_path):
        print(f"Datei nicht gefunden: {image_path}")
        return None

    img = cv2.imread(image_path)
    if img is None:
        print(f"Konnte Bild nicht laden: {image_path}")
        return None

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_color)
        if len(eyes) >= 2:
            return roi_color
    return None


In [16]:

path_to_cropped = "./dataset/cropped"
celebrity_file_names_dict = {}

for celebrity_folder in os.listdir(path_to_cropped):
    celeb_path = os.path.join(path_to_cropped, celebrity_folder)

    if os.path.isdir(celeb_path):
        celeb_name = celebrity_folder.lower().replace(" ", "_")
        image_paths = []

        for file in sorted(os.listdir(celeb_path)):
            image_path = os.path.join(celeb_path, file)
            image_paths.append(image_path)

        celebrity_file_names_dict[celeb_name] = image_paths



In [17]:

def w2d(img, mode = "haar", level =1):
    imArray = img

    imArray = cv2.cvtColor(imArray, cv2.COLOR_RGB2GRAY )
    imArray = np.float32(imArray)
    imArray /= 255
    
    coeffs = pywt.wavedec2(imArray, mode, level = level) 

    coeffs_H = list(coeffs)
    coeffs_H[0] *= 0

    imArray_H = pywt.waverec2(coeffs_H, mode)
    imArray_H *= 255;
    imArray_H = np.uint8(imArray_H)

    return imArray_H